# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [21]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

#Visualizing the first few rows of the dataset
print(crops.head())

#Checking if any column has null values
print(crops.isna().sum())

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice
N       0
P       0
K       0
ph      0
crop    0
dtype: int64


In [22]:
#Loading and Exploring data.
#Checking data structure of the dataframe
print(crops.head())
#Checking for data type and non-null counts
print(crops.info())
#Descriptive stats of the dataframe
print(crops.describe())
#Checking for missing values.
print(crops.isna().sum())

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None
                 N            P            K           ph
count  2200.000000  2200.000000  2200.000000  2200.000000
mean     50.551818    53.362727    48.149091     6.469480
std      36.917334    32.985883    50.647931     0.773938
min       0.000000     5.000000     5.000000     3.504752
25%      21.000000    28.000000    20.000000     5.971693
50%      37.000000    51.000000    32.0

In [23]:
#Encoding the crop column as categorical label
print("Original 'crop' column:")
print(crops['crop'].value_counts())

#Initializing label encoder
le = LabelEncoder()
crops['crop_encoded'] = le.fit_transform(crops['crop'])
print("\nDataFrame with encoded 'crop' column:")
print(crops[['crop', 'crop_encoded']])

print("\nMapping of original crop names to encoded labels:")
# You can get the mapping like this:
crop_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(crop_mapping)
print(crops.columns)

Original 'crop' column:
crop
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

DataFrame with encoded 'crop' column:
        crop  crop_encoded
0       rice            20
1       rice            20
2       rice            20
3       rice            20
4       rice            20
...      ...           ...
2195  coffee             5
2196  coffee             5
2197  coffee             5
2198  coffee             5
2199  coffee             5

[2200 rows x 2 columns]

Mapping of original crop names to encoded labels:
{'apple': np.int64(0), 'banana': np.int64(1), 'blackgram': np.int64(2), 'chickpea': np.i

In [29]:
#Splitting the data
X = crops[['N','P','K','ph']] #Features
y = crops['crop_encoded'] #Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 24)

#Initializing the scaler
scaler = StandardScaler()

#Fit the scaler on X_train and transform X_train
X_train_scaled = scaler.fit_transform(X_train)

#Convert scaled arrays back to DataFrame for easy column selection by name
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

#Transfrom x_test using the same fitted scaler
X_test_scaled = scaler.transform(X_test)

#Convert scaled arrays back to DataFrame for easy column selection by name
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [30]:
#Creating an empty dictionary to store each feature and its performance
features_dict = {}
#Looping through each feature
feature_list = ['N','P','K','ph']
for feature in feature_list:
    #Selecting current feature from scaled training data
    X_train_single_feature = X_train_scaled[[feature]].values
    #Selecting current feature from scaled test data
    X_test_single_feature = X_test_scaled[[feature]].values
    #Instantiating model 
    log_reg = LogisticRegression()
    #Fitting model to the training data
    log_reg.fit(X_train_single_feature, y_train)
    #Predicting the target values with the test data
    y_pred =log_reg.predict(X_test_single_feature)
    #Using the f1_score as the performance metric, with the average argument set to weighted
    feature_performance = accuracy_score(y_test, y_pred)
    features_dict[feature] = feature_performance
    
print(features_dict)

{'N': 0.12363636363636364, 'P': 0.16545454545454547, 'K': 0.23272727272727273, 'ph': 0.0890909090909091}


In [31]:
#Validating result
#Converting the features_dict into a dataframe and sorting the values based on the score in descending order.
result_df = pd.DataFrame(features_dict.items(), columns = ['Feature', 'Score']).sort_values('Score', ascending=False)
print(result_df)
#Subsetting and selecting the first row 
best_predictive_feature = {}
best_predictive_feature[result_df['Feature'].iloc[0]] = result_df['Score'].iloc[0]
print(best_predictive_feature)

  Feature     Score
2       K  0.232727
1       P  0.165455
0       N  0.123636
3      ph  0.089091
{'K': np.float64(0.23272727272727273)}
